In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from src.common.WebDriver import WebDriver
from src.common.utils import *
import json
import time
import re
from src.instagram.Instagram import Instagram
from src.instagram.UILogin import UILogin
from src.instagram.ApiCrawler import ApiCrawler

config = loadConfiguration('config/config.yaml')
credentials = loadCredentials('config/credentials.yaml')
logger = getLogger(__name__)
logger.debug(credentials)

02Apr2019 04:20:18 - [__main__] [DEBUG] - {'instagram': {'user': 'instafollowdev', 'password': 'password@123'}}


In [2]:
'''
Logging in to instagram using API is very difficult and they regularly change certain request/response headers for login.
API login would need for maintenance than UI version and their algorithm can more easily figure out it as bot.
Scrapping using selenium is very slow and impossible for account with large number of followers/following.
So, I found logging in using selenium UI automation and then scrapping data using API as best option.
'''

'\nLogging in to instagram using API is very difficult and they regularly change certain request/response headers for login.\nAPI login would need for maintenance than UI version and their algorithm can more easily figure out it as bot.\nScrapping using selenium is very slow and impossible for account with large number of followers/following.\nSo, I found logging in using selenium UI automation and then scrapping data using API as best option.\n'

In [3]:
crawler = ApiCrawler(credentials['instagram'], 'haileybieber')

In [4]:
crawler.getFollowingsByApi()

In [ ]:
print(crawler.followings)

In [ ]:
instagram = Instagram(credentials['instagram'], credentials['instagram']['user'])

In [ ]:
print(instagram.followings)

In [ ]:
driver = WebDriver().getDriver()
driver.get("https://www.instagram.com/accounts/login/")

In [ ]:
wait = WebDriverWait(driver, config['webdriver']['wait']['time'])

In [ ]:
logger.info("Logging in to instagram with user: "+credentials['instagram']['user'])
driver.get(config['instagram']['loginUrl'])

wait.until(EC.presence_of_element_located((By.XPATH, "//input[@name='username'][@type='text']")))
logger.debug("Presence of username field detected.")

userNameInput = driver.find_element_by_xpath("//input[@name='username'][@type='text']")
passwordInput = driver.find_element_by_xpath("//input[@name='password'][@type='password']")
userNameInput.send_keys(credentials['instagram']['user'])
passwordInput.send_keys(credentials['instagram']['password'])
driver.find_element_by_xpath("//button[@type='submit']").click()

In [ ]:
# finding user_id of logged in account
allCookies = driver.get_cookies()
currentUserId = -1
for cookie in allCookies:
    if(cookie['name']=='ds_user_id'):
        currentUserId = cookie['value']
        break
if currentUserId==-1:
    logger.error("Unable to retrieve ds_user_id cookie from cookies for logged in user {}".format(credentials['instagram']['user']))
logger.info("ds_user_id cookie of logged in user {}: {}".format(credentials['instagram']['user'], currentUserId))

In [ ]:
#getting id of an user
handle = "priyankachopra"
driver.get("view-source:https://www.instagram.com/{}/".format(handle))

In [ ]:
x = re.search("\"has_requested_viewer\":(.)*\"id\":(.)*\"is_business_account\":.", driver.find_element_by_xpath("//body").text)
print(x.group())
print(re.split(",", x.group()))

idOfTargetHandle = -1
for keyValue in re.split(",", x.group()):
    if "id" in keyValue:
        idOfTargetHandle = keyValue[6:-1]
if idOfTargetHandle==-1:
    logger.error("Unable to obtain id of target handle: {} from html source".format(handle))
else:
    logger.info("id of handle {} from html source: {}".format(handle, idOfTargetHandle))

In [ ]:
#API feature - get followings
# constructing url
queryVariables = {}
queryVariables["id"] = idOfTargetHandle
queryVariables["include_reel"] = config['instagram']['api']['include_reel']
queryVariables["fetch_mutual"] = config['instagram']['api']['fetch_mutual']
queryVariables["first"] = config['instagram']['api']['first']
queryVariablesJson = json.dumps(queryVariables).replace(" ", "")


queryUrl = config['instagram']['api']['url']
queryUrl += "?query_hash="+config['instagram']['api']['query_hash']['followings']
queryUrl += "&variables="+queryVariablesJson
#driver.get('https://www.instagram.com/graphql/query/?query_hash=56066f031e6239f35a904ac20c9f37d9&variables={"id":"4019903081","first":50}')
logger.info("QueryUrl: "+queryUrl)
driver.get(queryUrl)

In [ ]:
data = json.loads(driver.find_element_by_xpath("//body").text)
followings = []
followings.extend(data['data']['user']['edge_follow']['edges'])
print(len(followings))
#print(data['data']['user']['edge_followed_by'])

In [ ]:
print(data['data']['user']['edge_follow']['page_info']['has_next_page'])

while data['data']['user']['edge_follow']['page_info']['has_next_page']:
    queryVariables["after"] = data['data']['user']['edge_follow']['page_info']['end_cursor']
    queryVariablesJson = json.dumps(queryVariables..replace(" ", ""))
    queryUrl = config['instagram']['api']['url']
    queryUrl += "?query_hash="+config['instagram']['api']['query_hash']['followings']
    queryUrl += "&variables="+queryVariablesJson
    logger.info("QueryUrl: "+queryUrl)
    driver.get(queryUrl)
    data = json.loads(driver.find_element_by_xpath("//body").text)
    followings.extend(data['data']['user']['edge_follow']['edges'])
    print(len(followings))

In [ ]:
print(queryVariablesJson.replace(" ", ""))

In [ ]:
driver.get("view-source:https://www.instagram.com/iamtluangpuii/")

In [ ]:
# Manual scrolling UI
driver.get("https://instagram.com/"+'instafollowdev')
driver.get("https://instagram.com/"+'instafollowdev')

In [ ]:
followersButton = driver.find_element_by_xpath("//a[contains(@href, 'followers')]")
totalFollowers = int(followersButton.find_element_by_xpath("span").get_attribute("title").replace(',', ''))
logger.info("Total followers : "+str(totalFollowers))
followersButton.click()

In [ ]:
followersDialog = driver.find_element_by_xpath("//div[@role='dialog' and contains(., 'Followers')]")

In [ ]:
# Activating the popup dialog and then going to bottom of page
# wait for some-time for names to appear
try:
    element = WebDriverWait(followersDialog, config['webdriver']['wait']['time']).until(
        EC.presence_of_element_located((By.XPATH, ".//ul//li//div[text()]"))
    )
except Exception as e:
    logger.error("Error in locating name of any follower. This is done to send PAGE_DOWN down to scroll down. Error: "+repr(e))
finally:
    print("done")

followersCount = 0
previousCount = 0
scrollWithoutChangeCount = 0
scrollsWithoutChangeLimit = 10
while True:
    actions = ActionChains(driver)
    actions.send_keys_to_element(followersDialog.find_elements_by_xpath(".//ul//li//div[text()]")[-1], Keys.PAGE_DOWN) #activating scroll
    actions.perform()
    time.sleep(1)
    followersCount = len(followersDialog.find_elements_by_xpath(".//ul//li"))
    if followersCount == previousCount:
        scrollWithoutChangeCount += 1
    else:
        scrollWithoutChangeCount = 0
    if scrollWithoutChangeCount == scrollsWithoutChangeLimit:
        break
    previousCount = followersCount
    print(str(followersCount), end='\r')

In [ ]:
followers = followersDialog.find_elements_by_xpath(".//ul//li//a[@title]")
for follower in followers:
    print(follower.get_attribute('title'))

In [ ]:
closeDialogButton = driver.find_element_by_xpath("//*[@aria-label='Close']")
closeDialogButton.click()

In [ ]:
followingButton = driver.find_element_by_xpath("//a[contains(@href, 'following')]")
totalFollowing = int(followingButton.find_element_by_xpath("span").text)
logger.info("totalFollowing="+str(totalFollowing))
followingButton.click()

In [ ]:
followingDialog = driver.find_element_by_xpath("//div[@role='dialog' and contains(., 'Following')]")

In [ ]:
# wait for some-time for names to appear
try:
    firstPersonHavingName = WebDriverWait(followingDialog, config['webdriver']['wait']['time']).until(
        EC.presence_of_element_located((By.XPATH, ".//ul//li//div[text()]"))
    )
except Exception as e:
    logger.error("Error in locating name of any follower. This is done to send PAGE_DOWN down to scroll down. Error: "+repr(e))
finally:
    print("done")

followingCount = 0
while(followingCount < totalFollowing):
    actions = ActionChains(driver)
    actions.send_keys_to_element(followingDialog.find_elements_by_xpath(".//ul//li//div[text()]")[-1], Keys.PAGE_DOWN) #activating scroll
    actions.perform()
    followingCount = len(followingDialog.find_elements_by_xpath(".//ul//li"))
    print(str(followingCount))
print(str(followingCount))
